## ***README***

There is a bug where the pyvista grid is removed when moving between time steps with the slider. A temporary fix is possible if you are using this notebook locally and have the developer version of Ubermag.

A few lines of code from the [pyvsita_field.py](https://github.com/ubermag/discretisedfield/blob/master/discretisedfield/plotting/pyvista_field.py) file will have to be commented out. These are:
- **Line 172**
- **Line 354**
- **Line 573**
- **Line 731**

These lines relate to the region bounds created with the pyvista plots.

When running the functions in the notebook click on the "toggle ruler" icon to show the grid.
***

# Pyvista Drive Demo

In Ubermag we can visualise micromagnetic (mm) systems with Holoviews using the function `drive.hv()`. `Drive.hv()` allows us to view the change in time of a mm system when a Time Driver simulation is run. 

We can also use Pyvista which is a 3D visualisation tool to also visual mm systems, however we currently do not have the ability to visualise all the time steps at once. Instead the user has to specify which time step they want to see by indexing the drive object as shown below:  

`Drive[-1].pyvista.vector()`

This notebook is a demo on how we can replicate the functionality of `drive.hv()` to Pyvista so that we can see all the timesteps of the mm system.

## System Setup

First we import our modules

In [ ]:
import pyvista as pv
import discretisedfield as df
import micromagneticmodel as mm
import micromagneticdata as mdata
import oommfc as oc
import panel as pn

pn.extension()

Below we initialise our system

In [ ]:
# Geometry
r = 50e-9  # Radius of the thin nano magnetic disk (m)
thickness = 10e-9  # sample thickness (m)

# Material (Permalloy) parameters
Ms = 1e5  # saturation magnetisation (A/m)
A = 13e-12  # exchange energy constant (J/m)

# Dynamics (LLG equation) parameters
gamma0 = mm.consts.gamma0  # gyromagnetic ratio (m/As)
alpha = 0.05  # Gilbert damping

system = mm.System(name="test_1")

# Energy equation. We omit Zeeman energy term, because H=0.
system.energy = mm.Exchange(A=A) + mm.Demag()

# Dynamics equation
system.dynamics = mm.Precession(gamma0=gamma0) + mm.Damping(alpha=alpha)


# initial magnetisation state
def m_init(point):
    x, y, _ = point
    c = 1e9  # (1/m)
    return (-c * y, c * x, 0.1)


# Defining the geometry of the material as a circular disk
def Ms_func(point):
    x, y, _ = point
    if x**2 + y**2 <= r**2:
        return Ms
    else:
        return 0


# Sample's centre is placed at origin
region = df.Region(p1=(-r, -r, -thickness / 2), p2=(r, r, thickness / 2))
mesh = df.Mesh(
    region=region, cell=(5e-9, 5e-9, 10e-9)
)  # default cell = (5e-9, 5e-9, 10e-9)

system.m = df.Field(mesh, nvdim=3, value=m_init, norm=Ms_func, valid="norm")

We run our `TimeDriver` simulation below for **5** ns and save our magnetisation in **50** steps

In [ ]:
H = (3.4e4, 0, 0)  # external magnetic field A/m

td = oc.TimeDriver()
td.drive(system, t=5e-9, n=50, verbose=2)

Running OOMMF (ExeOOMMFRunner):   0%|          | 0/100 files written [00:00]

Running OOMMF (ExeOOMMFRunner)[2025/04/05 21:08] took 33.0 s


Our example drive to be used for analysis

In [17]:
drive = mdata.Data(system.name)[-1]

## Functions

Below we have functions for four pyvista plotting functions. These are:

1. `drive.pyvista.vector()`
2. `drive.pyvista.volume()`
3. `drive.pyvista.contour()`
4. `drive.pyvista.streamlines()`

In [22]:
def drive_pv_vector(drive):
    """
    pyvista vector plot.

    This function visualises a vector field. We use a panel slider change the time step
    and a panel number indicator to label the corresponding index for the time step we are on.

    Parameters
    ----------
    drive:

        Drive object that is used for analysis
    """
    # Create a PyVista plotter
    plot = pv.Plotter()

    def time_pv(value):
        drive[value].pyvista.vector(name="x", plotter=plot)
        time_step.value = (drive.info["t"] / drive.n) * (value + 1)

    # Create a Panel slider
    slider = pn.widgets.IntSlider(
        name="Index", value=0, start=0, end=drive.n - 1, step=1
    )

    # Create a Panel FloatInput for the time step
    time_step = pn.indicators.Number(
        name="t:",
        format="{value:.2g}",
        title_size="10pt",
        font_size="10pt",
        default_color="white",
    )

    # Bind the slider to update function
    slider.param.watch(
        lambda event: time_pv(event.new), "value"
    )  # callback function where event.new is the newest slider value

    # Initialize with first value
    time_pv(slider.value)

    # Panel layout
    layout = pn.Row(
        slider,
        time_step,
        styles={
            "background": "black",
            "padding": "10px",
            "border-radius": "10px",
            "border": "1px solid white",
        },
    )

    display(layout)
    plot.show()

In [ ]:
def drive_pv_volume(drive):
    """
    pyvista volume plot.

    This function visualises the scalar field within a three-dimensional region by rendering a volume.
    We use a panel slider change the time step and a panel number indicator to label the corresponding
    index for the time step we are on.

    Parameters
    ----------
    drive:

        Drive object that is used for analysis
    """
    # Create a PyVista plotter
    plot = pv.Plotter()

    def time_pv(value):
        drive[value].pyvista.volume(name="x", plotter=plot)
        time_step.value = (drive.info["t"] / drive.n) * (value + 1)

    # Create a Panel slider
    slider = pn.widgets.IntSlider(
        name="Index", value=0, start=0, end=drive.n - 1, step=1
    )

    # Create a Panel FloatInput for the time step
    time_step = pn.indicators.Number(
        name="t:",
        format="{value:.2g}",
        title_size="10pt",
        font_size="10pt",
        default_color="white",
    )

    # Bind the slider to update function
    slider.param.watch(
        lambda event: time_pv(event.new), "value"
    )  # callback function where event.new is the newest slider value

    # Initialize with first value
    time_pv(slider.value)

    # Panel layout
    layout = pn.Row(
        slider,
        time_step,
        styles={
            "background": "black",
            "padding": "10px",
            "border-radius": "10px",
            "border": "1px solid white",
        },
    )

    display(layout)
    plot.show()

In [ ]:
def drive_pv_contour(drive, isosurfaces=10):
    """
    pyvista contour plot.

    This function computes isosurfaces of the field. We use a panel slider
    change the time step and a panel number indicator to label the corresponding
    index for the time step we are on.

    Parameters
    ----------
    drive:

        Drive object that is used for analysis

    isosurfaces:

        Number of isosurfaces. Defaults to 10
    """
    # Create a PyVista plotter
    plot = pv.Plotter()

    def time_pv(value):
        drive[value].pyvista.contour(name="x", plotter=plot, isosurfaces=isosurfaces)
        time_step.value = (drive.info["t"] / drive.n) * (value + 1)

    # Create a Panel slider
    slider = pn.widgets.IntSlider(
        name="Index", value=0, start=0, end=drive.n - 1, step=1
    )

    # Create a Panel FloatInput for the time step
    time_step = pn.indicators.Number(
        name="t:",
        format="{value:.2g}",
        title_size="10pt",
        font_size="10pt",
        default_color="white",
    )

    # Bind the slider to update function
    slider.param.watch(
        lambda event: time_pv(event.new), "value"
    )  # callback function where event.new is the newest slider value

    # Initialize with first value
    time_pv(slider.value)

    # Panel layout
    layout = pn.Row(
        slider,
        time_step,
        styles={
            "background": "black",
            "padding": "10px",
            "border-radius": "10px",
            "border": "1px solid white",
        },
    )

    display(layout)
    plot.show()

In [ ]:
def drive_pv_streamlines(drive):
    """
    pyvista streamline plot.

    This function generates a plot of streamines. We use a
    panel slider change the time step and a panel number
    indicator to label the correspondingindex for the time
    step we are on.

    Parameters
    ----------
    drive:

        Drive object that is used for analysis
    """
    # Create a PyVista plotter
    plot = pv.Plotter()

    def time_pv(value):
        drive[value].pyvista.streamlines(name="x", plotter=plot)
        time_step.value = (drive.info["t"] / drive.n) * (value + 1)

    # Create a Panel slider
    slider = pn.widgets.IntSlider(
        name="Index", value=0, start=0, end=drive.n - 1, step=1
    )

    # Create a Panel FloatInput for the time step
    time_step = pn.indicators.Number(
        name="t:",
        format="{value:.2g}",
        title_size="10pt",
        font_size="10pt",
        default_color="white",
    )

    # Bind the slider to update function
    slider.param.watch(
        lambda event: time_pv(event.new), "value"
    )  # callback function where event.new is the newest slider value

    # Initialize with first value
    time_pv(slider.value)

    # Panel layout
    layout = pn.Row(
        slider,
        time_step,
        styles={
            "background": "black",
            "padding": "10px",
            "border-radius": "10px",
            "border": "1px solid white",
        },
    )

    display(layout)
    plot.show()

## Code examples

In the cells below we have an example of our `drive_pv_vector()` function and the holoviews equivalent, `drive.hv()`, to compare it with.

In [ ]:
drive_pv_vector(drive)

In [ ]:
drive.hv(kdims=["x", "y"])